***
# 08. 순환 신경망(Recurrent Neural Network)
***

앞서 배운 피드 포워드 신경망은 입력의 길이가 고정되어 있어 자연어 처리를 위한 신경망으로는 한계가 있었습니다. 결국 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요하게 되었는데, 자연어 처리에 대표적으로 사용되는 인공 신경망이 바로 순환 신경망(Recurrent Neural Network, RNN)입니다. 이번 챕터에서는 가장 기본적인 순환 신경망인 바닐라 RNN, 이를 개선한 LSTM, GRU에 대해서 학습해봅시다. LSTM과 GRU를 이해한다면 텍스트 분류나 기계 번역과 같은 다양한 자연어 처리 문제들을 풀 수 있습니다.

## 08-01 순환 신경망(Recurrent Neural Network, RNN)
***
RNN(Recurrent Neural Network)은 입력과 출력을 시퀀스 단위로 처리하는 시퀀스(Sequence) 모델입니다. 번역기를 생각해보면 입력은 번역하고자 하는 단어의 시퀀스인 문장입니다. 출력에 해당되는 번역된 문장 또한 단어의 시퀀스입니다. 이와 같이 시퀀스들을 처리하기 위해 고안된 모델들을 시퀀스 모델이라고 합니다. 그 중 RNN은 가장 기본적인 인공 신경망 시퀀스 모델입니다.

뒤에서 배우는 LSTM이나 GRU 또한 근본적으로 RNN에 속합니다. RNN을 이해하고 'RNN을 이용한 텍스트 분류' 챕터, '태깅 작업' 챕터, 'RNN을 이용한 인코더-디코더' 챕터에서 실습을 진행합니다.

용어는 비슷하지만 순환 신경망과 재귀 신경망(Recursive Neural Network)은 전혀 다른 개념입니다.  

### 1. 순환 신경망(Recurrent Neural Network, RNN)
***

앞서 배운 신경망들은 전부 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향했습니다. 이와 같은 신경망들을 피드 포워드 신경망(Feed Forward Neural Network)이라고 합니다. 그런데 그렇지 않은 신경망들이 있습니다. RNN(Recurrent Neural Network) 또한 그 중 하나입니다. RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있습니다.

![그림](img.png)

위의 그림을 보겠습니다. $x$는 입력층의 입력 벡터, $y$는 출력층의 출력 벡터입니다.
실제로는 편향 $b$도 입력으로 존재할 수 있지만 앞으로의 그림에서는 생략합니다.
RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 합니다.
이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현합니다.

은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다.
앞으로는 현재 시점을 변수 t로 표현하겠습니다. 이는 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다.
그렇다면 메모리 셀이 갖고 있는 이 값은 뭐라고 부를까요?

메모리 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값을 은닉 상태(hidden state) 라고 합니다.
다시 말해 t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.

![그림](img_1.png)

RNN을 표현할 때는 일반적으로 위의 그림에서 좌측과 같이 화살표로 사이클을 그려서 재귀 형태로 표현하기도 하지만,
우측과 같이 사이클을 그리는 화살표 대신 여러 시점으로 펼쳐서 표현하기도 합니다. 두 그림은 동일한 그림으로 단지 사이클을 그리는 화살표를 사용하여 표현하였느냐,
시점의 흐름에 따라서 표현하였느냐의 차이일 뿐 둘 다 동일한 RNN을 표현하고 있습니다.

피드 포워드 신경망에서는 뉴런이라는 단위를 사용했지만, RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터와 출력 벡터,
은닉층에서는 은닉 상태라는 표현을 주로 사용합니다. 위의 그림에서 회색과 초록색으로 표현한 각 네모들은 기본적으로 벡터 단위를 가정하고 있습니다.
피드 포워드 신경망과의 차이를 비교하기 위해서 RNN을 뉴런 단위로 시각화해보겠습니다.

![그림](img_2.png)

위의 그림은 입력 벡터의 차원이 4, 은닉 상태의 크기가 2, 출력층의 출력 벡터의 차원이 2인 RNN이 시점이 2일 때의 모습을 보여줍니다.
다시 말해 뉴런 단위로 해석하면 입력층의 뉴런 수는 4, 은닉층의 뉴런 수는 2, 출력층의 뉴런 수는 2입니다.

![그림](img_3.png)

RNN은 입력과 출력의 길이를 다르게 설계 할 수 있으므로 다양한 용도로 사용할 수 있습니다. 위 그림은 입력과 출력의 길이에 따라서 달라지는 RNN의 다양한 형태를 보여줍니다. 위 구조가 자연어 처리에서 어떻게 사용될 수 있는지 예를 들어봅시다. RNN 셀의 각 시점의 입, 출력의 단위는 사용자가 정의하기 나름이지만 가장 보편적인 단위는 '단어 벡터'입니다.

예를 들어 하나의 입력에 대해서 여러개의 출력을 의미하는 일 대 다(one-to-many) 구조의 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용할 수 있습니다. 사진의 제목은 단어들의 나열이므로 시퀀스 출력입니다.

![그림](img_4.png)

또한 단어 시퀀스에 대해서 하나의 출력을 하는 다 대 일(many-to-one) 구조의 모델은 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification),
또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스팸 메일 분류(spam detection) 등에 사용할 수 있습니다. 위 그림은 RNN으로 스팸 메일을 분류할 때의 아키텍처를 보여줍니다.
이러한 예제들은 'RNN을 이용한 텍스트 분류' 챕터에서 배웁니다.

![그림](img_5.png)

다 대 다(many-to-many) 구조의 모델의 경우에는 사용자가 문장을 입력하면 대답 문장을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기,
또는 '태깅 작업' 챕터에서 배우는 개체명 인식이나 품사 태깅과 같은 작업이 속합니다. 위 그림은 개체명 인식을 수행할 때의 RNN 아키텍처를 보여줍니다.

RNN에 대한 수식을 정의해보겠습니다.

![그림](img_6.png)

현재 시점 t에서의 은닉 상태값을 
$ht$라고 정의하겠습니다. 은닉층의 메모리 셀은 
$ht$를 계산하기 위해서 총 두 개의 가중치를 가집니다. 하나는 입력층을 위한 가중치 
$Wx$이고, 하나는 이전 시점 t-1의 은닉 상태값인 
$ht-1$을 위한 가중치 
$Wh$입니다.

이를 식으로 표현하면 다음과 같습니다.

![그림](img_7.png)

RNN의 은닉층 연산을 벡터와 행렬 연산으로 이해해봅시다. 자연어 처리에서 RNN의 입력 
$xt$는 대부분의 경우 단어 벡터로 간주할 수 있는데, 단어 벡터의 차원을 
$d$라고 하고, 은닉 상태의 크기를 
$Dh$라고 하였을 때 각 벡터와 행렬의 크기는 다음과 같습니다.

![그림](img_8.png)

배치 크기가 1이고, 
$d$와 $Dh$두 값 모두를 4로 가정하였을 때, RNN의 은닉층 연산을 그림으로 표현하면 아래와 같습니다.

![그림](img_9.png)

이때 $ht$
를 계산하기 위한 활성화 함수로는 주로 하이퍼볼릭탄젠트 함수(tanh)가 사용됩니다.
위의 식에서 각각의 가중치 $Wx$, $Wh$, $Wy$의 값은 하나의 층에서는 모든 시점에서 값을 동일하게 공유합니다.
하지만 은닉층이 2개 이상일 경우에는 각 은닉층에서의 가중치는 서로 다릅니다.

출력층은 결과값인 $yt$를 계산하기 위한 활성화 함수로는 푸는 문제에 따라서 다를텐데,
예를 들어서 이진 분류를 해야하는 경우라면 출력층에 로지스틱 회귀를 사용하여 시그모이드 함수를 사용할 수 있고
다중 클래스 분류를 해야하는 경우라면 출력층에 소프트맥스 회귀를 사용하여로 소프트맥스 함수를 사용할 수 있습니다.

### 2. 케라스(Keras)로 RNN 구현하기
***
케라스로 RNN 층을 추가하는 코드는 다음과 같습니다.

In [2]:
from tensorflow.keras.models import Sequential

hidden_units = 32  # 예시로 32로 설정
timesteps = 10  # 예시로 10으로 설정
input_dim = 8  # 예시로 8로 설정
M = 10
N = 8

model = Sequential()

In [3]:
from tensorflow.keras.layers import SimpleRNN

model.add(SimpleRNN(hidden_units))

인자를 사용할 때를 보겠습니다.

In [8]:
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))

- **hidden_units** = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. 
RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.
- **timesteps** = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.
- **input_dim** = 입력의 크기.

![그림](img_10.png)

RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로 받습니다.
batch_size는 한 번에 학습하는 데이터의 개수를 말합니다. 여기서부터는 텐서의 개념을 반드시 이해해야 하므로 벡터와 행렬 연산 챕터의 텐서 설명을 참고하시기 바랍니다.
다만, 이러한 표현은 사람이나 문헌에 따라서, 또는 풀고자 하는 문제에 따라서 종종 다르게 기재되는데 의 그림은 문제와 상황에 따라서 다르게 표현되는 입력 3D 텐서의 대표적인 표현들을 보여줍니다.

주의할 점은 위 코드는 출력층까지 포함한 인공 신경망 코드가 아니라 주로 은닉층으로 간주할 수 있는 하나의 RNN 층에 대한 코드입니다.
해당 코드가 리턴하는 결과값은 하나의 은닉 상태 또는 정의하기에 따라 여러 개의 시점의 은닉 상태 입니다.
아래의 그림은 만약 전결합층(Fully-connected layer)을 출력층으로 사용하였을 경우의 인공 신경망 그림과 은닉층까지만 표현한 그림의 차이를 보여줍니다.

![그림](img_11.png)

그렇다면 RNN 층은 위에서 설명한 입력 3D 텐서를 입력받아서 어떻게 은닉 상태를 출력할까요? RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력을 내보냅니다.
메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴합니다.
하지만, 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴합니다.
이는 RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정이 가능합니다. output_dim은 앞서 코드에서 정의한 hidden_units의 값으로 설정됩니다.

![그림](img_12.png)

위의 그림은 time step=3일 때, return_sequences = True를 설정했을 때와 그렇지 않았을 때 어떤 차이가 있는지를 보여줍니다.
return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력하며,
별도 기재하지 않거나 return_sequences=False로 선택할 경우에는 메모리 셀은 하나의 은닉 상태값만을 출력합니다.
그리고 이 하나의 값은 마지막 시점(time step)의 메모리 셀의 은닉 상태값입니다.

마지막 은닉 상태만 전달하도록 하면 다 대 일(many-to-one) 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면,
다음층에 RNN 은닉층이 하나 더 있는 경우이거나 다 대 다(many-to-many) 문제를 풀 수 있습니다.

뒤에서 배우는 LSTM이나 GRU도 내부 메커니즘은 다르지만 model.add()를 통해서 층을 추가하는 코드는 SimpleRNN 코드와 같은 형태를 가집니다.
실습을 통해 모델 내부적으로 출력 결과를 어떻게 정의하는지 이해해봅시다.

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 3)                 42        
                                                                 
Total params: 42 (168.00 Byte)
Trainable params: 42 (168.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


출력값이 (batch_size, output_dim) 크기의 2D 텐서일 때,
output_dim은 hidden_units의 값인 3입니다. 이 경우 batch_size를 현 단계에서는 알 수 없으므로 (None, 3)이 됩니다.
이번에는 batch_size를 미리 정의해보겠습니다.

In [10]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_7 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42 (168.00 Byte)
Trainable params: 42 (168.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


batch_size를 8로 기재하면 출력의 크기가 (8, 3)이 됩니다.
return_sequences 매개 변수에 True를 기재하여 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하도록 모델을 만들어 보겠습니다.

In [11]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42 (168.00 Byte)
Trainable params: 42 (168.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


출력의 크기가 (8, 2, 3)이 됩니다.

### 3. 파이썬으로 RNN 구현하기
***

직접 Numpy로 RNN 층을 구현해보겠습니다. 앞서 메모리 셀에서 은닉 상태를 계산하는 식을 다음과 같이 정의하였습니다.

![그림](img_13.png)

실제 구현에 앞서 간단히 가상의 코드(pseudocode)를 작성해보겠습니다.

```
# 아래의 코드는 가상의 코드(pseudocode)로 실제 동작하는 코드가 아님. 

hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받는다.
    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.
```

우선 t 시점의 은닉 상태를 hidden_state_t라는 변수로 선언하였고, 입력 데이터의 길이를 input_length로 선언하였습니다.
이 경우, 입력 데이터의 길이는 곧 총 시점의 수(timesteps)가 됩니다. 그리고 t 시점의 입력값을 input_t로 선언하였습니다.
각 메모리 셀은 각 시점마다 input_t와 hidden_sate_t(이전 상태의 은닉 상태)를 입력으로 활성화 함수인 하이퍼볼릭탄젠트 함수를 통해 현 시점의 hidden_state_t를 계산합니다.

가상의 코드를 통해 간단히 개념 정립을 해보았습니다. RNN 층을 실제 동작되는 코드로 구현해보겠습니다.
아래의 코드는 이해를 돕기 위해 (timesteps, input_dim) 크기의 2D 텐서를 입력으로 받았다고 가정하였으나,
실제로 케라스에서는 (batch_size, timesteps, input_dim)의 크기의 3D 텐서를 입력으로 받는 것을 기억합시다.

timesteps는 시점의 수입니다. 자연어 처리에서는 보통 문장의 길이입니다.
input_dim은 입력의 차원입니다. 자연어 처리에서는 보통 단어 벡터의 차원입니다. hidden_units는 은닉 상태의 크기로 메모리 셀의 용량입니다.
초기 은닉 상태는 0의 값을 가지는 벡터로 초기화합니다. 초기 은닉 상태를 출력해보겠습니다.

In [12]:
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,)) 

print('초기 은닉 상태 :',hidden_state_t)


초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


은닉 상태의 크기를 8로 정의하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터가 출력됩니다.
가중치와 편향을 각 크기에 맞게 정의하고 크기를 출력해보겠습니다.

In [13]:
Wx = np.random.random((hidden_units, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_units, hidden_units)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_units,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

print('가중치 Wx의 크기(shape) :',np.shape(Wx))
print('가중치 Wh의 크기(shape) :',np.shape(Wh))
print('편향의 크기(shape) :',np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


각 가중치와 편향의 크기는 다음과 같습니다. Wx는 (은닉 상태의 크기 × 입력의 차원), Wh는 (은닉 상태의 크기 × 은닉 상태의 크기), b는 (은닉 상태의 크기)의 크기를 가집니다.
이제 모든 시점의 은닉 상태를 출력한다고 가정하고, RNN 층을 동작시켜봅시다.

In [14]:
total_hidden_states = []

# 각 시점 별 입력값.
for input_t in inputs:

  # Wx * Xt + Wh * Ht-1 + b(bias)
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b)

  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
  # 각 시점의 은닉 상태의 값을 계속해서 누적
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도.
total_hidden_states = np.stack(total_hidden_states, axis = 0) 

# (timesteps, output_dim)
print('모든 시점의 은닉 상태 :')
print(total_hidden_states)

모든 시점의 은닉 상태 :
[[0.80391446 0.86834143 0.85764737 0.91008318 0.51830353 0.83934155
  0.98091778 0.96534425]
 [0.99984269 0.99974302 0.99976639 0.99882364 0.99967892 0.99877968
  0.99999736 0.99991036]
 [0.99990821 0.99981453 0.99993513 0.99917845 0.99982178 0.99765399
  0.99999766 0.99991854]
 [0.99987955 0.99983257 0.99983218 0.99883418 0.99991018 0.99847675
  0.99999577 0.99988348]
 [0.99986066 0.99978357 0.99983185 0.9989331  0.99990375 0.99893061
  0.99999749 0.99992688]
 [0.99994045 0.99993191 0.99984744 0.99924116 0.99994979 0.99899738
  0.99999679 0.99988017]
 [0.99996905 0.99995712 0.99992395 0.99954034 0.99995178 0.99925306
  0.99999863 0.99993347]
 [0.99996812 0.99991867 0.99996278 0.99953491 0.99992911 0.99951286
  0.99999955 0.9999845 ]
 [0.99997972 0.99996694 0.9999448  0.99959914 0.99996463 0.99956214
  0.99999924 0.99996774]
 [0.99996774 0.99995671 0.99988056 0.99939418 0.99997518 0.99967615
  0.99999882 0.99996316]]


### 4. 깊은 순환 신경망(Deep Recurrent Neural Network)
***

![그림](img_14.png)

앞서 RNN도 다수의 은닉층을 가질 수 있다고 언급한 바 있습니다.
위의 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 보여줍니다.
은닉층을 2개 추가하는 경우 코드는 아래와 같습니다.

In [15]:
model = Sequential()
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))

위의 코드에서 첫번째 은닉층은 다음 은닉층이 존재하므로 return_sequences = True를 설정하여 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내주고 있습니다.

### 5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)
***

양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라,
이후 시점의 입력 또한 예측에 기여할 수 있다는 아이디어에 기반합니다. 빈칸 채우기 문제에 비유하여 보겠습니다.

운동을 열심히 하는 것은 [        ]을 늘리는데 효과적이다.

1) 근육
2) 지방
3) 스트레스

'운동을 열심히 하는 것은 [ ]을 늘리는데 효과적이다.' 라는 문장에서 문맥 상으로 정답은 '근육'입니다.
위의 빈 칸 채우기 문제를 풀 때 이전에 나온 단어들만으로 빈 칸을 채우려고 시도해보면 정보가 부족합니다.
'운동을 열심히 하는 것은' 까지만 주고 뒤의 단어들은 가린 채 빈 칸의 정답이 될 수 있는 세 개의 선택지 중 고르는 것은 뒤의 단어들까지 알고있는 상태보다 명백히 정답을 결정하기가 어렵습니다.

RNN이 풀고자 하는 문제 중에서는 과거 시점의 입력 뿐만 아니라 미래 시점의 입력에 힌트가 있는 경우도 많습니다.
그래서 이전과 이후의 시점 모두를 고려해서 현재 시점의 예측을 더욱 정확하게 할 수 있도록 고안된 것이 양방향 RNN입니다.

![그림](img_15.png)

양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용합니다.
첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States) 를 전달받아 현재의 은닉 상태를 계산합니다.
위의 그림에서는 주황색 메모리 셀에 해당됩니다. 두번째 메모리 셀은 앞에서 배운 것과는 다릅니다.
앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States) 를 전달 받아 현재의 은닉 상태를 계산합니다.
입력 시퀀스를 반대 방향으로 읽는 것입니다. 위의 그림에서는 초록색 메모리 셀에 해당됩니다. 그리고 이 두 개의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해 사용됩니다.

In [16]:
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))

양방향 RNN도 다수의 은닉층을 가질 수 있습니다.
아래의 그림은 양방향 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 양방향 순환 신경망의 모습을 보여줍니다.

![그림](img_16.png)

다른 인공 신경망 모델들도 마찬가지이지만,
은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다. 은닉층을 추가하면 학습할 수 있는 양이 많아지지만 반대로 훈련 데이터 또한 많은 양이 필요합니다.
아래의 코드는 은닉층이 4개인 경우를 보여줍니다.

In [17]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))

양방향 RNN은 태깅 작업 챕터의 실습에서 사용해보겠습니다.

### 6. 점검 퀴즈
***

RNN을 제대로 이해했는지 퀴즈를 통해서 확인해보세요! 모델에 대한 설명이 다음과 같을 때, 총 파라미터 개수를 구해보세요.

1. Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100입니다.
2. 은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128입니다.
3. 훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.
4. 이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.
5. 은닉층은 1개입니다.

> **1. Embedding 레이어 파라미터 수: 단어 집합 크기 × 임베딩 벡터의 차원**
  > 5000 × 100 = 500,000

> **2. Simple RNN 레이어 파라미터 수:**
  > 입력에서 은닉층으로의 가중치: 입력 차원 × 은닉 상태의 크기 
  은닉층에서 은닉층으로의 가중치: 은닉 상태의 크기 × 은닉 상태의 크기
  편향(bias): 은닉 상태의 크기
  (입력 차원 = 임베딩 벡터의 차원, 은닉 상태의 크기 = 128)
  입력에서 은닉층으로의 가중치: 100 × 128 = 12,800
  은닉층에서 은닉층으로의 가중치: 128 × 128 = 16,384
  편향: 128

> **3. 출력층 파라미터 수:**
  >입력 차원 × 출력 차원 + 편향
  입력 차원 = 은닉 상태의 크기, 출력 차원 = 1
  128 × 1 + 1 = 129

따라서 총 파라미터 수는 각 레이어의 파라미터 수를 합산한 것입니다.

500,000 (Embedding) + 12,800 (Simple RNN 입력에서 은닉층) + 16,384 (Simple RNN 은닉층에서 은닉층) + 128 (Simple RNN 편향) + 129 (출력층) = 529,441

따라서 총 파라미터 수는 529,441개입니다.

## 08-02 장단기 메모리(Long Short-Term Memory, LSTM)
***

바닐라 아이스크림이 가장 기본적인 맛을 가진 아이스크림인 것처럼, 앞서 배운 RNN을 가장 단순한 형태의 RNN이라고 하여 바닐라 RNN(Vanilla RNN)이라고 합니다.
(케라스에서는 SimpleRNN) 바닐라 RNN 이후 바닐라 RNN의 한계를 극복하기 위한 다양한 RNN의 변형이 나왔습니다.
이번에 배우게 될 LSTM도 그 중 하나입니다. 앞으로의 설명에서 LSTM과 비교하여 RNN을 언급하는 것은 전부 바닐라 RNN을 말합니다.

### 1. 바닐라 RNN의 한계
***

![그림](img_17.png)

앞에서 바닐라 RNN은 출력 결과가 이전의 계산 결과에 의존한다는 것을 언급한 바 있습니다.
하지만 바닐라 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점이 있습니다.
바닐라 RNN의 시점(time step)이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생합니다.
위의 그림은 첫번째 입력값인 $x1$의 정보량을 짙은 남색으로 표현했을 때,
색이 점차 얕아지는 것으로 시점이 지날수록 $x1$의 정보량이 손실되어가는 과정을 표현하였습니다.
뒤로 갈수록 $x1$의 정보량은 손실되고,
시점이 충분히 긴 상황에서는 $x1$의 전체 정보에 대한 영향력은 거의 의미가 없을 수도 있습니다.

어쩌면 가장 중요한 정보가 시점의 앞 쪽에 위치할 수도 있습니다.
RNN으로 만든 언어 모델이 다음 단어를 예측하는 과정을 생각해봅시다.
예를 들어 "모스크바에 여행을 왔는데 건물도 예쁘고 먹을 것도 맛있었어. 그런데 글쎄 직장 상사한테 전화가 왔어. 어디냐고 묻더라구 그래서 나는 말했지. 저 여행왔는데요. 여기 ___"
다음 단어를 예측하기 위해서는 장소 정보가 필요합니다. 그런데 장소 정보에 해당되는 단어인 '모스크바'는 앞에 위치하고 있고, RNN이 충분한 기억력을 가지고 있지 못한다면 다음 단어를 엉뚱하게 예측합니다.

이를 **장기 의존성 문제(the problem of Long-Term Dependencies)** 라고 합니다.

### 2. 바닐라 RNN 내부 열어보기
***

![그림](img_18.png)

LSTM에 대해서 이해해보기 전에 바닐라 RNN의 뚜껑을 열어보겠습니다. 위의 그림은 바닐라 RNN의 내부 구조를 보여줍니다.
이 책에서는 RNN 계열의 인공 신경망의 그림에서는 편향 $b$를 생략합니다.
위의 그림에 편향 $b$를 그린다면 $xt$옆에 tanh로 향하는 또 하나의 입력선을 그리면 됩니다.

![그림](img_19.png)

바닐라 RNN은 $xt$와 $ht-1$이라는 두 개의 입력이 각각의 가중치와 곱해져서 메모리 셀의 입력이 됩니다.
그리고 이를 하이퍼볼릭탄젠트 함수의 입력으로 사용하고 이 값은 은닉층의 출력인 은닉 상태가 됩니다.

### 3. LSTM(Long Short-Term Memory)
***

![그림](img_20.png)

위의 그림은 LSTM의 전체적인 내부의 모습을 보여줍니다. 전통적인 RNN의 이러한 단점을 보완한 RNN의 일종을 장단기 메모리(Long Short-Term Memory)라고 하며, 줄여서 LSTM이라고 합니다.
LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정합니다.
요약하면 LSTM은 은닉 상태(hidden state)를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태(cell state)라는 값을 추가하였습니다.
위의 그림에서는 t시점의 셀 상태를 $Ct$로 표현하고 있습니다. LSTM은 RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보입니다.

![그림](img_21.png)

셀 상태는 위의 그림에서 왼쪽에서 오른쪽으로 가는 굵은 선입니다. 셀 상태 또한 이전에 배운 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용됩니다.

은닉 상태의 값과 셀 상태의 값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용합니다.
각 게이트는 삭제 게이트, 입력 게이트, 출력 게이트라고 부르며 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재합니다.
시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는데 이 값들을 가지고 게이트를 조절합니다. 아래의 내용을 참고로 각 게이트에 대해서 알아보겠습니다.

![그림](img_22.png)

#### (1) 입력 게이트

![그림](img_23.png)

입력 게이트는 현재 정보를 기억하기 위한 게이트입니다.
우선 현재 시점 t의 $x$값과 입력 게이트로 이어지는 가중치 $Wxi$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $Whi$를 곱한 값을 더하여 시그모이드 함수를 지납니다.
이를 $it$라고 합니다.
그리고 현재 시점 t의 $x$값과 입력 게이트로 이어지는 가중치 $Wxg$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $Whg$를 곱한 값을 더하여 하이퍼볼릭탄젠트 함수를 지납니다.
이를 $gt$라고 합니다.
시그모이드 함수를 지나 0과 1사이의 값을 가지는 $it$과 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값을 가지는 $gt$.
이 두 개의 값을 가지고 이번에 선택된 기억할 정보의 양을 정하는데, 구체적으로 어떻게 결정하는지는 아래에서 배우게 될 셀 상태 수식을 참고합니다.

#### (2) 삭제 게이트

![그림](img_24.png)

삭제 게이트는 기억을 삭제하기 위한 게이트입니다. 현재 시점 t의 $x$값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지나게 됩니다.
시그모이드 함수를 지나면 0과 1 사이의 값이 나오게 되는데, 이 값이 곧 삭제 과정을 거친 정보의 양입니다. 0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것입니다.
이를 가지고 셀 상태를 구하게 되는데, 구체적으로는 아래에서 배우게 될 셀 상태 수식을 참고합니다.

#### (3) 셀 상태

![그림](img_25.png)

셀 상태 $Ct$를 구하는 방법을 알아보겠습니다.
삭제 게이트에서 일부 기억을 잃은 상태입니다.

입력 게이트에서 구한 $it$, $gt$이 두 개의 값에 대해서 원소별 곱(entrywise product)을 진행합니다.
다시 말해 같은 크기의 두 행렬이 있을 때 같은 위치의 성분끼리 곱하는 것을 말합니다.
여기서는 식으로 ○로 표현합니다. 이것이 이번에 선택된 기억할 값입니다.

입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더합니다.
이 값을 현재 시점 t의 셀 상태라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨집니다.

삭제 게이트와 입력 게이트의 영향력을 이해해봅시다. 만약 삭제 게이트의 출력값인 $ft$가 0이 된다면,
이전 시점의 셀 상태의 값인 $Ct-1$은 현재 시점의 셀 상태의 값을 결정하기 위한 영향력이 0이 되면서,
오직 입력 게이트의 결과만이 현재 시점의 셀 상태의 값 $Ct$을 결정할 수 있습니다.
이는 삭제 게이트가 완전히 닫히고 입력 게이트를 연 상태를 의미합니다. 반대로 입력 게이트의 $it$값을 0이라고 한다면,
현재 시점의 셀 상태의 값 $Ct$는 오직 이전 시점의 셀 상태의 값 $Ct-1$의 값에만 의존합니다.
이는 입력 게이트를 완전히 닫고 삭제 게이트만을 연 상태를 의미합니다.
결과적으로 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정합니다.

#### (4) 출력 게이트와 은닉 상태

![그림](img_26.png)

출력 게이트는 현재 시점 t의 $x$값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값입니다.
해당 값은 현재 시점 t의 은닉 상태를 결정하는 일에 쓰이게 됩니다.
셀 상태의 값이 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값이 되고, 해당 값은 출력 게이트의 값과 연산되면서,
값이 걸러지는 효과가 발생하여 은닉 상태가 됩니다. 은닉 상태의 값은 또한 출력층으로도 향합니다.

## 08-03 게이트 순환 유닛(Gated Recurrent Unit, GRU)
***

GRU(Gated Recurrent Unit)는 2014년 뉴욕대학교 조경현 교수님이 집필한 논문에서 제안되었습니다.
GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄였습니다.
다시 말해서, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화 시켰습니다.

### 1. GRU(Gated Recurrent Unit)
***
LSTM에서는 출력, 입력, 삭제 게이트라는 3개의 게이트가 존재했습니다. 반면, GRU에서는 업데이트 게이트와 리셋 게이트 두 가지 게이트만이 존재합니다.
GRU는 LSTM보다 학습 속도가 빠르다고 알려져있지만 여러 평가에서 GRU는 LSTM과 비슷한 성능을 보인다고 알려져 있습니다.

![그림](img_27.png)

GRU와 LSTM 중 어떤 것이 모델의 성능면에서 더 낫다라고 단정지어 말할 수 없으며,
기존에 LSTM을 사용하면서 최적의 하이퍼파라미터를 찾아낸 상황이라면 굳이 GRU로 바꿔서 사용할 필요는 없습니다.

경험적으로 데이터 양이 적을 때는 매개 변수의 양이 적은 GRU가 조금 더 낫고,
데이터 양이 더 많으면 LSTM이 더 낫다고도 합니다. GRU보다 LSTM에 대한 연구나 사용량이 더 많은데, 이는 LSTM이 더 먼저 나온 구조이기 때문입니다.

### 2. 케라스에서의 GRU(Gated Recurrent Unit)
***

케라스에서는 역시 GRU에 대한 구현을 지원합니다. 사용 방법은 SimpleRNN이나 LSTM과 같습니다.
```
model.add(GRU(hidden_size, input_shape=(timesteps, input_dim)))
```